In [1]:
import boto3
import json
from glom import glom, T
import sqlalchemy as sa
from sqlalchemy.engine.url import URL

In [30]:
workgroup_name = 'wg-streaming'
namespace_name = 'ns-streaming'
secret_name = 'REDSHIFT_PASSWORD'
admin_username = 'admin'
db_name = 'dev'
port = 5439

In [31]:
redshift_client = boto3.client('redshift-serverless')
secretsmanager_client = boto3.client('secretsmanager')
endpoint = redshift_client.get_workgroup(workgroupName=workgroup_name)['workgroup']['endpoint']['address']
admin_password = secretsmanager_client.get_secret_value(
    SecretId=secret_name,
    )['SecretString']

In [32]:
redshift_url = URL.create(
drivername='redshift+redshift_connector', # indicate redshift_connector driver and dialect will be used
host=endpoint, # Amazon Redshift host
port=port, # Amazon Redshift port
database=db_name, # Amazon Redshift database
username=admin_username,
password=admin_password
)

In [41]:
%reload_ext sql
%config SqlMagic.displaylimit = 25
%config SqlMagic.displaycon = False
%sql $redshift_url

In [42]:
result_set = %sql select current_user
user_df.DataFrame()

Done.


,current_user
0,admin


In [36]:
%%sql
select current_user

Done.


current_user
admin
